In [8]:
from typing import Annotated
from typing_extensions import TypedDict, Optional

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import add_messages
from langchain_deepseek import ChatDeepSeek
from langchain_ollama import ChatOllama

from pydantic import BaseModel, Field

llm = ChatOllama(model="qwen2.5:0.5b")

message = [HumanMessage(content="Hello, world!")]

llm.invoke(message)

AIMessage(content="Hello! How can I help you today? Is there something in particular you'd like to know or discuss? Feel free to ask me anything you're curious about!", additional_kwargs={}, response_metadata={'model': 'qwen2.5:0.5b', 'created_at': '2025-06-21T12:02:22.4860564Z', 'done': True, 'done_reason': 'stop', 'total_duration': 347709700, 'load_duration': 19361200, 'prompt_eval_count': 33, 'prompt_eval_duration': 8716100, 'eval_count': 34, 'eval_duration': 317513600, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run--601834a4-a6cb-40e3-bad5-ca6c9a7fa586-0', usage_metadata={'input_tokens': 33, 'output_tokens': 34, 'total_tokens': 67})

In [ ]:


class userinfo(BaseModel):
    name: str = Field(..., description="The name of the user")
    id: str = Field(..., description="The id of the user, 身份证号")
    phone: Optional[str] = Field(None, description="The phone number of the user")
    introduction: Optional[str] = Field(None, description="A brief introduction of the user")


structurt = llm.with_structured_output(userinfo)

# answer = structurt.invoke("My name is John Doe, my id is 123456789012345678, my phone number is 123-456-7890, and I am a software engineer.")


class input_state(TypedDict):
    # mm: Annotated[int, add_messages]
    mm: int


def node_a(input:input_state):
    """A node that processes the input state."""
    x = input["mm"]
    print(input)
    return {"mm": x+10}

def node_b(input:input_state):
    """A node that processes the input state."""
    x = input["mm"]
    print(input)
    return {"mm": x*10}

graph = StateGraph(input_state)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)

graph.add_edge(START, "node_a")
graph.add_edge("node_a", "node_b")
graph.add_edge("node_b", END)

graph = graph.compile()

answer = graph.invoke({'mm': 23})
print(answer)

In [ ]:
class input_state(TypedDict):
    mm: Annotated[list[AnyMessage], add_messages]
    # mm: int


def node_a(input:input_state):
    """A node that processes the input state."""
    message = {"role": "user", "content": "what's your name?"}
    answer = llm.invoke(message)
    return {"mm": answer}

def node_b(input:input_state):
    """A node that processes the input state."""
    x = input["mm"]
    messages = input["mm"]
    return {"mm": x*10}

graph = StateGraph(input_state)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)

graph.add_edge(START, "node_a")
graph.add_edge("node_a", "node_b")
graph.add_edge("node_b", END)

graph = graph.compile()

answer = graph.invoke({'mm': 23})
print(answer)